In [1]:
class Square(object):
    def __init__(self, side):
        self.side = side
    def aget(self):
        return self.side ** 2
    def aset(self, value):
        print('Cannot set area')
    def adel(self):
        print('Cannot delete area')
    area = property(aget, aset, adel, doc='Area of a square.')

In [2]:
s = Square(5)

In [3]:
s.area

25

In [5]:
!cd ..

In [6]:
ls

Untitled.ipynb  descriptors/    metaclasses/    properties/


In [ ]:
# %load properties/square.py
"""Properties with decorators.
"""

from __future__ import print_function

class Square(object):
    """A square using properties with decorators.
    """

    def __init__(self, side):
        self.side = side

    @property
    def area(self):
        """Calculate the area of the square when the
        attribute is accessed."""
        return self.side * self.side

    @area.setter
    def area(self, value):
        """Don't allow setting."""
        print("Can't set the area")

    @area.deleter
    def area(self):
        """Don't allow deleting."""
        print("Can't delete the area.")


if __name__ == '__main__':

    square = Square(5)
    print('area:', square.area)
    print('try to set')
    square.area = 10
    print('area:', square.area)
    print('try to delete')
    del square.area
    print('area:', square.area)


In [9]:
# %load properties/square_nested.py
"""Use a decorator to allow nested properties.
"""

from __future__ import print_function

def nested_property(func):
    """Make defining properties simpler.
    """
    names = func()
    # We want the docstring from the decorated function.
    # If we do not set 'doc', we get the docstring from `fget`.
    names['doc'] = func.__doc__
    return property(**names)


class Square(object):
    """A square using properties with decorators.
    """

    def __init__(self, side):
        self.side = side

    @nested_property
    def area():
        """Property that defines is methods nested.
        """

        def fget(self):
            """
            Calculate the area of the square
            when the attribute is accessed.
            """
            return self.side * self.side

        def fset(self, value):
            """Don't allow setting."""
            print("Can't set the area")

        def fdel(self):
            """Don't allow deleting."""
            print("Can't delete the area.")

        return locals()


if __name__ == '__main__':

    square = Square(5)
    print('area:', square.area)
    print('try to set')
    square.area = 10
    print('area:', square.area)
    print('try to delete')
    del square.area
    print('area:', square.area)
    print(Square.area.__doc__)


area: 25
try to set
Can't set the area
area: 25
try to delete
Can't delete the area.
area: 25
Property that defines is methods nested.
        


In [10]:
ls

Untitled.ipynb  descriptors/    metaclasses/    properties/


In [12]:
# %load descriptors/datadescriptor.py
"""A typical data descriptor.
"""

from __future__ import print_function

class DataDescriptor(object):
    """A simple descriptor.
    """
    def __init__(self):
        self.value = 0
    def __get__(self, instance, cls):
        print('data descriptor __get__')
        return self.value
    def __set__(self, instance, value):
        print('data descriptor __set__')
        try:
            self.value = value.upper()
        except AttributeError:
            self.value = value
    def __delete__(self, instance):
        print("Don't like to be deleted." )


In [13]:
class A(object):
    attr = DataDescriptor()

In [14]:
a = A()

In [15]:
a.attr

data descriptor __get__


0

The above internally does:

In [16]:
type(a).__dict__['attr'].__get__(a, type(a))

data descriptor __get__


0

In [17]:
a.attr = 'hi'

data descriptor __set__


In [18]:
a.attr

data descriptor __get__


'HI'

In [19]:
"""A typical non data descriptor.
"""

from __future__ import print_function

class NonDataDescriptor(object):
    """A simple descriptor.
    """
    def __init__(self):
        self.value = 0
    def __get__(self, instance, cls):
        print('data descriptor __get__')
        return self.value + 10

In [20]:
class B(object):
    attr = NonDataDescriptor()

In [21]:
ten = B()

In [22]:
ten.attr

data descriptor __get__


10

In [23]:
ten.attr = 100

In [24]:
ten.attr

100

In [26]:
class Overridden(object):
    attr = DataDescriptor()
    def __getattribute__(self, name):
        print('no way')

In [28]:
o = Overridden()

In [29]:
o.attr

no way


In [30]:
def func():
    pass

In [31]:
func.__get__

<method-wrapper '__get__' of function object at 0x1054ebb90>

In [32]:
class C(object):
    def meth(self):
        pass

In [33]:
c = C()

In [34]:
c.meth

<bound method C.meth of <__main__.C object at 0x105029410>>

In [35]:
C.meth

<unbound method C.meth>

In [37]:
# %load descriptors/class_storage.py

"""A descriptor works only in a class.

Storing attribute data directly in a descriptor
means sharing between instances.
"""

from __future__ import print_function


class DescriptorClassStorage(object):
    """Descriptor storing data in class."""

    def __init__(self, default=None):
        self.value = default

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        self.value = value


if __name__ == '__main__':
    class StoreClass(object):
        """All instances will share `attr`.
        """
        attr = DescriptorClassStorage(10)

    store1 = StoreClass()
    store2 = StoreClass()
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('Setting store1 only.')
    store1.attr = 100
    print('store1', store1.attr)
    print('store2', store2.attr)


store1 10
store2 10
Setting store1 only.
store1 100
store2 100


In [39]:
# %load descriptors/weakkeydict_storage.py

"""A descriptor works only in a class.

We can store a different value for each instance in a dictionary
in the descriptor.
"""

from __future__ import print_function

from weakref import WeakKeyDictionary


class DescriptorWeakKeyDictStorage(object):
    """Descriptor that stores attribute data in instances.
    """
    _hidden = WeakKeyDictionary()

    def __init__(self, default=None):
        self.default = default

    def __get__(self, instance, owner):
        return DescriptorWeakKeyDictStorage._hidden.get(instance, self.default)

    def __set__(self, instance, value):
        DescriptorWeakKeyDictStorage._hidden[instance] = value


if __name__ == '__main__':
    class StoreInstance(object):
        """All instances have own `attr`.
        """
        attr = DescriptorWeakKeyDictStorage(10)

    store1 = StoreInstance()
    store2 = StoreInstance()
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('Setting store1 only.')
    store1.attr = 100
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('_hidden:', DescriptorWeakKeyDictStorage._hidden.items())
    del store1
    print('Deleted store1')
    print('_hidden:', DescriptorWeakKeyDictStorage._hidden.items())



store1 10
store2 10
Setting store1 only.
store1 100
store2 10
_hidden: [(<__main__.StoreInstance object at 0x105507c50>, 100)]
Deleted store1
_hidden: []


In [41]:
# %load descriptors/checked.py

"""Example for descriptor that checks conditions on attributes.
"""
from __future__ import print_function

from weakref import WeakKeyDictionary


class Checked(object):
    """Descriptor that checks with a user-supplied check function
    if an attribute is valid.
    """

    _hidden = WeakKeyDictionary()

    def __init__(self, checker=None, default=None):
        if checker:
            # checker must be a callable
            checker(default)
        self.checker = checker
        self.default = default

    def __get__(self, instance, owner):
        return Checked._hidden.get(instance, self.default)

    def __set__(self, instance, value):
        if self.checker:
            self.checker(value)
        Checked._hidden[instance] = value


if __name__ == '__main__':

    def is_int(value):
        """Check if value is an integer.
        """
        if not isinstance(value, int):
            raise ValueError('Int required {} found'.format(type(value)))

    class Restricted(object):
        """Use checked attributes.
        """
        attr1 = Checked(checker=is_int, default=10)
        attr2 = Checked(default=12.5)
        # Setting the default to float, `is_int` raises a `ValueError`.
        try:
            attr3 = Checked(checker=is_int, default=12.5)
        except ValueError:
            print('Cannot set default to float, must be int.')
            attr3 = Checked(checker=is_int, default=12)

    restricted = Restricted()
    print('attr1', restricted.attr1)
    restricted.attr1 = 100
    print('attr1', restricted.attr1)
    try:
        restricted.attr1 = 200.12
    except ValueError:
        print('Cannot set attr1 to float, must be int.')
        restricted.attr1 = 200


Cannot set default to float, must be int.
attr1 10
attr1 100
Cannot set attr1 to float, must be int.


In [42]:
class C(object):
    attr = 10

same as:

In [43]:
C = type('C', (object,), {'attr': 10})

In [44]:
class MyMeta(type):
    def __str__(cls):
        return 'hello'

In [45]:
class A:
    __metaclass__ = MyMeta

In [46]:
str(A)

'hello'

In [47]:
A

__main__.A

In [49]:
type(A)

__main__.MyMeta

In [50]:
a = A

In [56]:
class MyMeta(type):
    def __new__(mcl, name, bases, cdict):
        print(mcl)
        print(name)
        print(bases)
        print(cdict)
        super(MyMeta, mcl).__new__(mcl, name, bases, cdict)

In [57]:
class New:
    __metaclass__ = MyMeta

<class '__main__.MyMeta'>
New
()
{'__module__': '__main__', '__metaclass__': <class '__main__.MyMeta'>}


Interesting...

In [59]:
# %load metaclasses/meta_2_3.py
# file: meta_2_3.py

"""
The code is a bit hard to understand. The basic idea is exploiting the idea
that metaclasses can customize class creation and are picked by by the parent
class. This particular implementation uses a metaclass to remove its own parent
from the inheritance tree on subclassing. The end result is that the function
creates a dummy class with a dummy metaclass. Once subclassed the dummy
classes metaclass is used which has a constructor that basically instances a
new class from the original parent and the actually intended metaclass.
That way the dummy class and dummy metaclass never show up.

From:
 http://lucumr.pocoo.org/2013/5/21/porting-to-python-3-redux/#metaclass-syntax-changes

Used in:

* Jinja2
* SQLAlchemy
* future (python-future.org)

"""

from __future__ import print_function
import platform


# from jinja2/_compat.py
def with_metaclass(meta, *bases):
    # This requires a bit of explanation: the basic idea is to make a
    # dummy metaclass for one level of class instanciation that replaces
    # itself with the actual metaclass.  Because of internal type checks
    # we also need to make sure that we downgrade the custom metaclass
    # for one level to something closer to type (that's why __call__ and
    # __init__ comes back from type etc.).
    #
    # This has the advantage over six.with_metaclass in that it does not
    # introduce dummy classes into the final MRO.
    class metaclass(meta):
        __call__ = type.__call__
        __init__ = type.__init__

        def __new__(cls, name, this_bases, d):
            if this_bases is None:
                return type.__new__(cls, name, (), d)
            return meta(name, bases, d)

    return metaclass('temporary_class', None, {})


if __name__ == '__main__':

    class BaseClass(object):
        pass


    class MetaClass(type):
        """Metaclass for Python 2 and 3.
        """
        def __init__(cls, name, bases, cdict):
            print('It works with {impl} version {ver}.'.format(
                impl=platform.python_implementation(),
                ver=platform.python_version()))
            super(MetaClass, cls).__init__(name, bases, cdict)


    class Class(with_metaclass(MetaClass, BaseClass)):
        # BaseClass is optional.
        pass


It works with CPython version 2.7.5.


In [63]:
# %load metaclasses/noclassattr.py
#file: noclassattr.py

"""Preventing non-callable class attributes with a metaclass.
"""

from __future__ import print_function

class NoClassAttributes(type):
    """No non-callable class attributes allowed
    """
    def __init__(cls, name, bases, cdict):
        allowed = set(['__module__', '__metaclass__', '__doc__',
                       '__qualname__'])
        for key, value in cdict.items():
            if (key not in allowed) and (not callable(value)):
                msg = 'Found non-callable class attribute "%s". ' % key
                msg += 'Only methods are allowed.'
                raise Exception(msg)
        super(NoClassAttributes, cls).__init__(name, bases, cdict)


if __name__ == '__main__':

    from meta_2_3 import with_metaclass

    class AttributeChecker(with_metaclass(NoClassAttributes)):
        """Base class for meta.
        """
        pass

    class AttributeLess(AttributeChecker):
        """Only methods work.
        """
        def meth(self):
            """This is allowed'
            """
            print('Hello from AttributeLess.')

    attributeless = AttributeLess()
    attributeless.meth()


    class WithAttribute(AttributeChecker):
        """Has non-callable class attribute.
        Will raise an exception.
        """
        a = 10
        def meth(self):
            """This is allowed'
            """
            print('Hello from WithAttribute')



Hello from AttributeLess.


Exception: Found non-callable class attribute "a". Only methods are allowed.

In [62]:
cd metaclasses/

/Users/jordancounts/Desktop/PyCon/deco_meta/metaclasses


Can achieve same results using decorators:

In [65]:
# %load class_deco.py
# file: class_deco.py

def noclassattr_deco(cls):
    """Class decorator to allow only callable attributes.
    """
    allowed = set(['__module__', '__metaclass__', '__doc__', '__qualname__',
                   '__weakref__', '__dict__'])
    for key, value in cls.__dict__.items():
        if (key not in allowed) and (not callable(value)):
            msg = 'Found non-callable class attribute "%s". ' % key
            msg += 'Only methods are allowed.'
            raise Exception(msg)
    return cls


if __name__ == '__main__':

    @noclassattr_deco
    class AttributeLess(object):
        """Only methods work.
        """
        def meth(self):
            """This is allowed'
            """
            print('Hello from AttributeLess.')

    attributeless = AttributeLess()
    attributeless.meth()

    @noclassattr_deco
    class WithAttribute(object):
        """Has non-callable class attribute.
        Will raise an exception.
        """
        a = 10
        def meth(self):
            """This is allowed'
            """
            print('Hello from WithAttribute')

Hello from AttributeLess.


Exception: Found non-callable class attribute "a". Only methods are allowed.

In [67]:
# %load slotstyped.py
# file: slotstyped.py

"""Use of descriptor and metaclass to get slots with
given types.
"""

from __future__ import print_function

class TypDescriptor(object):
    """Descriptor with type.
    """

    def __init__(self, data_type, default_value=None):
        self.name = None
        self.data_type = data_type
        if default_value:
            self.default_value = default_value
        else:
            self.default_value = data_type()

    def __get__(self, instance, cls):
        return getattr(instance, self.name, self.default_value)

    def __set__(self, instance, value):
        if not isinstance(value, self.data_type):
            raise TypeError('Required data type is %s. Got %s' % (
            self.data_type, type(value)))
        setattr(instance, self.name, value)

    def __delete__(self, instance):
        raise AttributeError('Cannot delete %r' % instance)


class TypeProtected(type):
    """Metaclass to save descriptor values in slots.
    """

    def __new__(mcl, name, bases, cdict):
        slots = []
        for key, value in cdict.items():
            if isinstance(value, TypDescriptor):
                value.name = '__' + key
                slots.append(value.name)
        cdict['__slots__'] = slots
        return super(TypeProtected, mcl).__new__(mcl, name, bases, cdict)


if __name__ == '__main__':

    from meta_2_3 import with_metaclass


    class Typed(with_metaclass(TypeProtected)):
        pass

    class MyClass(Typed):
        """Test class."""
        attr1 = TypDescriptor(int)
        attr2 = TypDescriptor(float, 5.5)


    def main():
        """Test it.
        """
        my_inst = MyClass()
        print(my_inst.attr1)
        print(my_inst.attr2)
        print(dir(my_inst))
        print(my_inst.__slots__)
        my_inst.attr1 = 100.0
        print(my_inst.attr1)
        # this will fail
        my_inst.unknown = 100

    main()


0
5.5
['_MyClass__attr1', '_MyClass__attr2', '__class__', '__delattr__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'attr1', 'attr2']
['__attr2', '__attr1']


TypeError: Required data type is <type 'int'>. Got <type 'float'>

Slots example:

In [68]:
class S(object):
    __slots__ = ['a', 'b']

In [69]:
s = S()

In [70]:
s.a = 12

In [71]:
s.x = 15

AttributeError: 'S' object has no attribute 'x'

In [73]:
s.__dict__

AttributeError: 'S' object has no attribute '__dict__'